In [4]:
from PIL import Image, ImageChops
import numpy as np
import os
import cv2
import matplotlib.pyplot as plt
import random

from torchvision.transforms import ToPILImage
from skimage.metrics import structural_similarity as SSIM
from skimage.metrics import mean_squared_error as MSE
to_pil = ToPILImage()


In [5]:
def get_ssim_score(img, img_rec):
    ssim_score = SSIM(img, img_rec)
    return ssim_score

def get_mse_dif(img, img_rec):
    mse_error = MSE(img, img_rec)
    return mse_error

In [19]:
# Ground truth reconstruction comparision for mse and ssim

eval_dir = "../evaluation_output"
orig_dataset_dir = "../data/eval"

diff_ssim_sum = 0
diff_mse_sum = 0
for idx, seq_dir in enumerate(os.listdir(eval_dir)):
    gt_path = os.path.join(orig_dataset_dir, seq_dir, "gt.png")
    gt_img = np.array(Image.open(gt_path).resize((256, 256)))[:, :, :3]

    gt_rec_path = os.path.join(eval_dir, seq_dir, "gt_reconstruction.png")
    gt_rec_img = np.array(Image.open(gt_rec_path))

    gt_img_gray = np.array(Image.fromarray(gt_img).convert("L"))
    gt_rec_img_gray = np.array(Image.fromarray(gt_rec_img).convert("L"))

    diff_mse = get_mse_dif(gt_img, gt_rec_img)
    diff_mse_sum += diff_mse
    
    ssim_score = get_ssim_score(gt_img_gray, gt_rec_img_gray)
    diff_ssim_sum += ssim_score
    # plt.show()
    
diff_ssim_sum /= 100
diff_mse_sum /= 100
print("Average ssim score: ", diff_ssim_sum)
print("Average mse error: ", diff_mse_sum)



Average ssim score:  0.6154662590151313
Average mse error:  3478.5573129272457


In [30]:
# Random ground truth comparision

eval_dir = "../evaluation_output"
orig_dataset_dir = "../data/eval"

diff_ssim_sum = 0
diff_mse_sum = 0

for idx, seq_dir in enumerate(os.listdir(eval_dir)):
    n= len(os.listdir(eval_dir)) -1
    
    random_idx = random.randint(0, n)
    while random_idx == idx:
        random_idx = random.randint(0, n)
    
    gt_path = os.path.join(orig_dataset_dir, seq_dir, "gt.png")
    gt_img = np.array(Image.open(gt_path).resize((256, 256)))[:, :, :3]
    
    random_seq_dir = os.listdir(eval_dir)[random_idx]
    random_gt_path = os.path.join(orig_dataset_dir, random_seq_dir, "gt.png")
    random_gt_img = np.array(Image.open(random_gt_path).resize((256, 256)))[:, :, :3]
    
    gt_img_gray = np.array(Image.fromarray(gt_img.astype(np.uint8)).convert("L"))
    random_gt_img_gray = np.array(Image.fromarray(random_gt_img.astype(np.uint8)).convert("L"))
    
    diff_ssim = get_ssim_score(gt_img_gray, random_gt_img_gray)
    diff_ssim_sum += diff_ssim
    
    diff_mse = get_mse_dif(gt_img, random_gt_img)
    diff_mse_sum += diff_mse


    
diff_mse_sum /= 100
diff_ssim_sum /= 100
print("Average mse error: ", diff_mse_sum)
print("Average SSIM error: ", diff_ssim_sum)

rand:  41
rand:  4
rand:  53
rand:  46
rand:  53
rand:  87
rand:  95
rand:  19
rand:  56
rand:  50
rand:  21
rand:  7
rand:  88
rand:  0
rand:  59
rand:  53
rand:  15
rand:  94
rand:  53
rand:  7
rand:  93
rand:  85
rand:  3
rand:  66
rand:  95
rand:  0
rand:  55
rand:  77
rand:  52
rand:  51
rand:  9
rand:  39
rand:  26
rand:  19
rand:  8
rand:  79
rand:  46
rand:  89
rand:  80
rand:  4
rand:  33
rand:  74
rand:  25
rand:  93
rand:  91
rand:  52
rand:  83
rand:  56
rand:  60
rand:  48
rand:  11
rand:  89
rand:  11
rand:  21
rand:  12
rand:  57
rand:  23
rand:  38
rand:  68
rand:  2
rand:  57
rand:  66
rand:  32
rand:  20
rand:  58
rand:  50
rand:  94
rand:  42
rand:  49
rand:  42
rand:  68
rand:  54
rand:  65
rand:  11
rand:  58
rand:  32
rand:  77
rand:  52
rand:  61
rand:  68
rand:  40
rand:  44
rand:  61
rand:  24
rand:  63
rand:  95
rand:  8
rand:  72
rand:  82
rand:  54
rand:  28
rand:  76
rand:  33
rand:  23
rand:  48
rand:  46
rand:  17
Average mse error:  7950.923075764976
Ave

In [18]:
# FINAL! comparision of occluded masked image with masked (gt_rec * shadow) + light
eval_dir = "../evaluation_output"
orig_dataset_dir = "../data/eval"

diff_ssim_sum = 0
diff_mse_sum = 0
count = 0
for idx, seq_dir in enumerate(os.listdir(eval_dir)):
    #print("Sequence number: ", seq_dir)
    gt_rec_path = os.path.join(eval_dir, seq_dir, "gt_reconstruction.png")
    gt_rec = np.array(Image.open(gt_rec_path).resize((256, 256)))
    
    gt_path = os.path.join(orig_dataset_dir, seq_dir, "gt.png")
    gt_img = np.array(Image.open(gt_path).resize((256, 256)))[:, :, :3]
    # if(seq_dir != "650"):
    #   continue
    for i in range(10):
        
        orig_occ_binary_path = os.path.join(orig_dataset_dir, seq_dir, str(i+1) + "-m.png")
        orig_occ_binary_img = np.array(Image.open(orig_occ_binary_path).resize((256, 256)))[:, :, :3]
        orig_occ_binary_img_inv = np.where(orig_occ_binary_img ==255, np.zeros(orig_occ_binary_img.shape), np.ones(orig_occ_binary_img.shape))
        
        ocl_path = os.path.join(orig_dataset_dir, seq_dir, str(i+1) + ".png")
        ocl_img = np.array(Image.open(ocl_path).resize((256, 256)))[:, :, :3]
        
        ocl_img_filtered = ocl_img * orig_occ_binary_img_inv

        shadow_i_path = os.path.join(eval_dir, seq_dir, "shadow_" + str(i+1) + ".png")
        shadow_i = np.array(Image.open(shadow_i_path)) / 255.0
        
        light_i_path = os.path.join(eval_dir, seq_dir, "light_" + str(i+1) + ".png")
        light_i = np.array(Image.open(light_i_path), dtype = np.int32)
        
        gt_rec_multi_add = np.array((gt_rec * shadow_i + light_i) * orig_occ_binary_img_inv, dtype=np.int32).clip(max = 255)
        

        gt_rec_multi_add_gray = np.array(Image.fromarray(gt_rec_multi_add.astype(np.uint8)).convert("L"))
        ocl_img_filtered_gray = np.array(Image.fromarray(ocl_img_filtered.astype(np.uint8)).convert("L"))
        
        ssim_score = get_ssim_score(ocl_img_filtered_gray, gt_rec_multi_add_gray)
        diff_ssim_sum += ssim_score
        mse_diff = get_mse_dif(ocl_img_filtered, gt_rec_multi_add)
        diff_mse_sum += mse_diff
        count += 1
        
#         fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(10, 4),
#                              sharex=True, sharey=True)

#         ax = axes.ravel()

#         ax[0].imshow(ocl_img_filtered_gray)
#         ax[0].set_title(f'Occluded image: {str(i+1)}')
#         ax[0].tick_params(left = False, right = False , labelleft = False , labelbottom = False, bottom = False)
        
#         ax[1].imshow(gt_rec_multi_add_gray)
#         ax[1].set_title('recon:')
#         ax[1].tick_params(left = False, right = False , labelleft = False , labelbottom = False, bottom = False)

 
#         plt.tight_layout()
#         plt.show()


print("SSIM: ", diff_ssim_sum / count)
print("MSE: ", diff_mse_sum / count)


SSIM:  0.9146761693473934
MSE:  121.52596048571401
